In [1]:
# Import useful libs
import dask
import numpy as np
import xarray as xr
import dask.array as da
import scipy.fft as sp_fft
from dask.distributed import Client, LocalCluster

In [2]:
# cluster = LocalCluster(n_workers=4)
client = Client()
client

c:\Users\baptiste.menetrier\.virtualenvs\phd-FBjz4hBd\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63648 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:63648/status,
Dashboard: http://127.0.0.1:63648/status,Workers: 4
Total threads: 12,Total memory: 15.64 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63651,Workers: 4
Dashboard: http://127.0.0.1:63648/status,Total threads: 12
Started: Just now,Total memory: 15.64 GiB
Comm: tcp://127.0.0.1:63673,Total threads: 3
Dashboard: http://127.0.0.1:63675/status,Memory: 3.91 GiB
Nanny: tcp://127.0.0.1:63654,


Load test case dataset 

In [3]:
fullpath = r"C:\Users\baptiste.menetrier\Desktop\devPy\phd\localisation\verlinden\verlinden_process_output\testcase1_0\ship\not_on_grid\hilbert_env_intercorr0\output_hilbert_env_intercorr0_noiseless.nc"
ds = xr.open_dataset(fullpath, chunks={'lon': 20, 'lat': 20})
# ds = ds.isel(lon=slice(0, 100), lat=slice(0, 100))
ds

<xarray.Dataset> Size: 4GB
Dimensions:              (idx_rcv: 2, lat: 171, lon: 171, idx_rcv_pairs: 1,
                          idx_rcv_in_pair: 2, library_signal_time: 8192,
                          library_corr_lags: 16383, event_signal_time: 8192,
                          src_trajectory_time: 10, event_corr_lags: 16383)
Coordinates:
  * lon                  (lon) float64 1kB -5.025 -5.023 ... -4.775 -4.774
  * lat                  (lat) float64 1kB 52.49 52.49 52.49 ... 52.64 52.64
  * idx_rcv              (idx_rcv) int32 8B 0 1
  * idx_rcv_pairs        (idx_rcv_pairs) int32 4B 0
  * idx_rcv_in_pair      (idx_rcv_in_pair) int32 8B 0 1
  * library_signal_time  (library_signal_time) float32 33kB 0.0 0.0025 ... 20.48
  * library_corr_lags    (library_corr_lags) int32 66kB -8191 -8190 ... 8191
  * event_signal_time    (event_signal_time) float32 33kB 0.0 0.0025 ... 20.48
  * src_trajectory_time  (src_trajectory_time) float32 40B 0.0 9.719 ... 87.47
  * event_corr_lags      (event_corr_lags) int32 66kB -8191 -8190 ... 8190 8191
Data variables: (12/16)
    lon_rcv              (idx_rcv) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
    lat_rcv              (idx_rcv) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
    r_from_rcv           (idx_rcv, lat, lon) float64 468kB dask.array<chunksize=(2, 20, 20), meta=np.ndarray>
    delay_rcv            (idx_rcv, lat, lon) float64 468kB dask.array<chunksize=(2, 20, 20), meta=np.ndarray>
    rcv_pairs            (idx_rcv_pairs, idx_rcv_in_pair) int32 8B dask.array<chunksize=(1, 2), meta=np.ndarray>
    rcv_signal_library   (idx_rcv, lat, lon, library_signal_time) float32 2GB dask.array<chunksize=(2, 20, 20, 8192), meta=np.ndarray>
    ...                   ...
    az_src_rcv           (idx_rcv, src_trajectory_time) float32 80B dask.array<chunksize=(2, 10), meta=np.ndarray>
    rcv_signal_event     (idx_rcv, src_trajectory_time, event_signal_time) float32 655kB dask.array<chunksize=(2, 10, 8192), meta=np.ndarray>
    event_corr           (idx_rcv_pairs, src_trajectory_time, event_corr_lags) float32 655kB dask.array<chunksize=(1, 10, 16383), meta=np.ndarray>
    ambiguity_surface    (idx_rcv_pairs, src_trajectory_time, lat, lon) float64 2MB dask.array<chunksize=(1, 10, 20, 20), meta=np.ndarray>
    detected_pos_lon     (idx_rcv_pairs, src_trajectory_time) float64 80B dask.array<chunksize=(1, 10), meta=np.ndarray>
    detected_pos_lat     (idx_rcv_pairs, src_trajectory_time) float64 80B dask.array<chunksize=(1, 10), meta=np.ndarray>
Attributes:
    title:               Verlinden simulation with simple environment
    dx:                  100
    dy:                  100
    fullpath_populated:  C:\Users\baptiste.menetrier\Desktop\devPy\phd\locali...
    snr_dB:              Noiseless
    source_positions:    Not interpolated on grid
    src_pos:             not_on_grid
    long_name:           Ambiguity surface
    units:               dB
    detection_metric:    hilbert_env_intercorr0
    fullpath_output:     C:\Users\baptiste.menetrier\Desktop\devPy\phd\locali...
    fullpath_analysis:   C:\Users\baptiste.menetrier\Desktop\devPy\phd\img\lo...

In [4]:
ds.rcv_signal_library

<xarray.DataArray 'rcv_signal_library' (idx_rcv: 2, lat: 171, lon: 171,
                                        library_signal_time: 8192)> Size: 2GB
dask.array<open_dataset-rcv_signal_library, shape=(2, 171, 171, 8192), dtype=float32, chunksize=(2, 20, 20, 8192), chunktype=numpy.ndarray>
Coordinates:
  * lon                  (lon) float64 1kB -5.025 -5.023 ... -4.775 -4.774
  * lat                  (lat) float64 1kB 52.49 52.49 52.49 ... 52.64 52.64
  * idx_rcv              (idx_rcv) int32 8B 0 1
  * library_signal_time  (library_signal_time) float32 33kB 0.0 0.0025 ... 20.48
Attributes:
    long_name:  $s_{i}$

In [5]:
ds.rcv_signal_library.sel(idx_rcv=0)

<xarray.DataArray 'rcv_signal_library' (lat: 171, lon: 171,
                                        library_signal_time: 8192)> Size: 958MB
dask.array<getitem, shape=(171, 171, 8192), dtype=float32, chunksize=(20, 20, 8192), chunktype=numpy.ndarray>
Coordinates:
  * lon                  (lon) float64 1kB -5.025 -5.023 ... -4.775 -4.774
  * lat                  (lat) float64 1kB 52.49 52.49 52.49 ... 52.64 52.64
    idx_rcv              int32 4B 0
  * library_signal_time  (library_signal_time) float32 33kB 0.0 0.0025 ... 20.48
Attributes:
    long_name:  $s_{i}$

In [6]:
def fft_convolve_f_dask(a0, a1, axis=-1):

    # Compute the cross-correlation of a0 and a1 using the FFT
    corr_01 = da.fft.irfft(a0 * np.conj(a1), axis=axis)
    # Reorganise so that tau = 0 corresponds to the center of the array
    nmid = corr_01.shape[-1] // 2 + 1
    corr_01 = np.concatenate((corr_01[..., nmid:], corr_01[..., :nmid]), axis=axis)
    return corr_01

In [7]:
%%time
# test with dask arrays
nchunck = 30
ax = 2
nlag = ds.sizes["library_corr_lags"]
nfft = sp_fft.next_fast_len(nlag, True)
library_corr_dim = ["idx_rcv_pairs", "lat", "lon", "library_corr_lags"]
library_corr = np.empty(tuple(ds.sizes[d] for d in library_corr_dim))

for i_pair in range(ds.sizes["idx_rcv_pairs"]):
    rcv_pair = ds.rcv_pairs.isel(idx_rcv_pairs=i_pair)
    in1 = ds.rcv_signal_library.sel(idx_rcv=rcv_pair[0])
    in2 = ds.rcv_signal_library.sel(idx_rcv=rcv_pair[1])

    # in1 = da.from_array(in1, chunks=(nchunck, nchunck, nlag))
    # in2 = da.from_array(in2, chunks=(nchunck, nchunck, nlag))

    sig_0 = da.fft.rfft(in1, nfft, axis=-1)
    sig_1 = da.fft.rfft(in2, nfft, axis=-1)
    
    corr_01_fft = fft_convolve_f_dask(sig_0, sig_1, axis=ax)
    corr_01_fft = corr_01_fft[:, :, slice(nlag)]

    autocorr0 = fft_convolve_f_dask(sig_0, sig_0, axis=ax)
    autocorr0 = autocorr0[:, :, slice(nlag)]

    autocorr1 = fft_convolve_f_dask(sig_1, sig_1, axis=ax)
    autocorr1 = autocorr1[:, :, slice(nlag)]

    n0 = corr_01_fft.shape[-1] // 2
    corr_norm = np.sqrt(autocorr0[..., n0] * autocorr1[..., n0])
    corr_norm = np.repeat(np.expand_dims(corr_norm, axis=ax), nlag, axis=ax)
    corr_01_fft /= corr_norm



CPU times: total: 188 ms
Wall time: 1.39 s


In [8]:
in1

<xarray.DataArray 'rcv_signal_library' (lat: 171, lon: 171,
                                        library_signal_time: 8192)> Size: 958MB
dask.array<getitem, shape=(171, 171, 8192), dtype=float32, chunksize=(20, 20, 8192), chunktype=numpy.ndarray>
Coordinates:
  * lon                  (lon) float64 1kB -5.025 -5.023 ... -4.775 -4.774
  * lat                  (lat) float64 1kB 52.49 52.49 52.49 ... 52.64 52.64
    idx_rcv              int32 4B 0
  * library_signal_time  (library_signal_time) float32 33kB 0.0 0.0025 ... 20.48
    idx_rcv_pairs        int32 4B 0
    idx_rcv_in_pair      int32 4B 0
Attributes:
    long_name:  $s_{i}$

In [9]:
# corr_01_fft = corr_01_fft.persist()

In [10]:
%%time
# sig_0, sig_1 = da.compute(sig_0, sig_1)
corr_01_fft = corr_01_fft.persist()
corr_01_fft_arr = corr_01_fft.compute()

CPU times: total: 51.8 s
Wall time: 2min 8s


In [11]:
def fft_convolve_f(a0, a1, axis=-1, workers=8):

    # Compute the cross-correlation of a0 and a1 using the FFT
    corr_01 = sp_fft.irfft(a0 * np.conj(a1), axis=axis, workers=workers)
    # Reorganise so that tau = 0 corresponds to the center of the array
    nmid = corr_01.shape[-1] // 2 + 1
    corr_01 = np.concatenate((corr_01[..., nmid:], corr_01[..., :nmid]), axis=axis)
    return corr_01

In [12]:
%%time
N_CORES=8

for i_pair in range(ds.sizes["idx_rcv_pairs"]):
    rcv_pair = ds.rcv_pairs.isel(idx_rcv_pairs=i_pair)

    in1 = ds.rcv_signal_library.sel(idx_rcv=rcv_pair[0]).values
    in2 = ds.rcv_signal_library.sel(idx_rcv=rcv_pair[1]).values

    nfft = sp_fft.next_fast_len(nlag, True)

    sig_0 = sp_fft.rfft(
        in1,
        n=nfft,
        axis=-1,
    )
    sig_1 = sp_fft.rfft(
        in2,
        n=nfft,
        axis=-1,
    )

    corr_01_fft = fft_convolve_f(sig_0, sig_1, axis=ax, workers=N_CORES)
    corr_01_fft = corr_01_fft[:, :, slice(nlag)]

    autocorr0 = fft_convolve_f(sig_0, sig_0, axis=ax, workers=N_CORES)
    autocorr0 = autocorr0[:, :, slice(nlag)]

    autocorr1 = fft_convolve_f(sig_1, sig_1, axis=ax, workers=N_CORES)
    autocorr1 = autocorr1[:, :, slice(nlag)]

    n0 = corr_01_fft.shape[-1] // 2
    corr_norm = np.sqrt(autocorr0[..., n0] * autocorr1[..., n0])
    corr_norm = np.repeat(np.expand_dims(corr_norm, axis=ax), nlag, axis=ax)
    corr_01_fft /= corr_norm
    # library_corr[i_pair, ...] = corr_01_fft

CPU times: total: 1min 6s
Wall time: 1min 37s


In [13]:
@dask.delayed
def task(ds, rcv_pair, nlag, ax, N_CORES=4):
    in1 = ds.rcv_signal_library.sel(idx_rcv=rcv_pair[0]).values
    in2 = ds.rcv_signal_library.sel(idx_rcv=rcv_pair[1]).values

    nfft = sp_fft.next_fast_len(nlag, True)

    sig_0 = sp_fft.rfft(
        in1,
        n=nfft,
        axis=-1,
    )
    sig_1 = sp_fft.rfft(
        in2,
        n=nfft,
        axis=-1,
    )

    corr_01_fft = fft_convolve_f(sig_0, sig_1, axis=ax)
    corr_01_fft = corr_01_fft[:, :, slice(nlag)]

    autocorr0 = fft_convolve_f(sig_0, sig_0, axis=ax)
    autocorr0 = autocorr0[:, :, slice(nlag)]

    autocorr1 = fft_convolve_f(sig_1, sig_1, axis=ax)
    autocorr1 = autocorr1[:, :, slice(nlag)]

    n0 = corr_01_fft.shape[-1] // 2
    corr_norm = np.sqrt(autocorr0[..., n0] * autocorr1[..., n0])
    corr_norm = np.repeat(np.expand_dims(corr_norm, axis=ax), nlag, axis=ax)
    corr_01_fft /= corr_norm

In [14]:
%%time
# Dealyed version 
N_CORES=4

for i_pair in range(ds.sizes["idx_rcv_pairs"]):
    rcv_pair = ds.rcv_pairs.isel(idx_rcv_pairs=i_pair)

    corr_01_fft = task(ds, rcv_pair, nlag, ax)
    # library_corr[i_pair, ...] = corr_01_fft

CPU times: total: 453 ms
Wall time: 3.47 s


In [15]:
corr_01_fft = corr_01_fft.persist()

In [16]:
corr_01_fft_ar = dask.compute(corr_01_fft)

2024-03-27 17:06:26,681 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:63679 (pid=13748) exceeded 95% memory budget. Restarting...
2024-03-27 17:06:29,976 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:63679' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('open_dataset-ambiguity_surface-5578a7e31371d4fa0f794fa707622bc3', 0, 0, 5, 1), ('open_dataset-rcv_signal_library-e5376b3fc675718dfd0f96fecf5353c1', 0, 5, 0, 0), ('open_dataset-library_corr-0d397f7c20dc90130f34d752d64e4363', 0, 2, 8, 0), ('open_dataset-r_from_rcv-fb9bff4d6637f4c72d5ad370e7203f5a', 0, 1, 7), ('open_dataset-delay_rcv-fb9bff4d6637f4c72d5ad370e7203f5a', 0, 2, 8), ('open_dataset-ambiguity_surface-5578a7e31371d4fa0f794fa707622bc3', 0, 0, 0, 7), ('open_dataset-rcv_signal_library-e5376b3fc675718dfd0f96fecf5353c1', 0, 7, 6, 0), ('open_dataset-r_from_rcv-fb9bff4d6637f4c72d5ad370e7203f5a', 0, 2, 8), ('open_dataset-delay_rcv-fb9bff4d6637f4c72d5ad370e